In [2]:
import torch
from torch_geometric.data import Data
import numpy as np
import scanpy as sc
import pandas as pd
from graph_construction import construct_graph
from tcn_autoencoder import train_and_extract_features
from data_splitting import create_train_val_tf, generate_train_val_kfold
from transformer_model import train_model, test_model
from data_splitting import split_edges, split_tfs


In [3]:
file_name = "mESC"
adata = sc.read("./dataset/" +file_name + ".h5ad")

filtered_refnet = adata.uns["grn"]
fea_df = pd.DataFrame(adata.uns["gpt_emb"])
fea_df["Gene"] = adata.var_names
X_norm = adata.X.T
tfs = np.unique(filtered_refnet["Gene1"])

In [4]:
data_orig, tfs_index, le = construct_graph(filtered_refnet, adata.var_names,fea_df, tfs)
data_orig

/home/alex/Documents/GitHub/GRNPT/graph_construction.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  edge_index = torch.tensor([le.transform(filtered_refnet['Gene1']),


Data(x=[500, 1536], edge_index=[2, 2347])

In [ ]:
# train the auto encoder and extract the features from it
tcauto_model, features = train_and_extract_features(X_norm,learning_rate=0.001, weight_decay=1e-4)
print(features.shape)


/home/alex/Documents/GitHub/GRNPT/.venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/200]
Epoch [2/200]
Epoch [3/200]
Epoch [4/200]
Epoch [5/200]
Epoch [6/200]
Epoch [7/200]
Epoch [8/200]
Epoch [9/200]
Epoch [10/200]
Epoch [11/200]
Epoch [12/200]
Epoch [13/200]
Epoch [14/200]
Epoch [15/200]
Epoch [16/200]
Epoch [17/200]
Epoch [18/200]
Epoch [19/200]
Epoch [20/200]
Epoch [21/200]
Epoch [22/200]
Epoch [23/200]
Epoch [24/200]
Epoch [25/200]
Epoch [26/200]
Epoch [27/200]
Epoch [28/200]
Epoch [29/200]
Epoch [30/200]
Epoch [31/200]
Epoch [32/200]
Epoch [33/200]
Epoch [34/200]
Epoch [35/200]
Epoch [36/200]
Epoch [37/200]
Epoch [38/200]
Epoch [39/200]
Epoch [40/200]
Epoch [41/200]
Epoch [42/200]
Epoch [43/200]
Epoch [44/200]
Epoch [45/200]
Epoch [46/200]
Epoch [47/200]
Epoch [48/200]
Epoch [49/200]
Epoch [50/200]
Epoch [51/200]
Epoch [52/200]
Epoch [53/200]
Epoch [54/200]
Epoch [55/200]
Epoch [56/200]
Epoch [57/200]
Epoch [58/200]
Epoch [59/200]
Epoch [60/200]
Epoch [61/200]
Epoch [62/200]
Epoch [63/200]
Epoch [64/200]
Epoch [65/200]
Epoch [66/200]
Epoch [67/200]
Epoc

In [6]:
reduced_features_df = pd.DataFrame(features.cpu().numpy())
reduced_features_df["Gene"] = adata.var_names
reduced_features_df

# create features of nodes
node_features_1 = fea_df.set_index('Gene').reindex(le.classes_).fillna(0).values
node_features_2 = reduced_features_df.set_index('Gene').reindex(le.classes_).fillna(0).values


x = torch.tensor(node_features_1, dtype=torch.float)
x_additional = torch.tensor(node_features_2, dtype=torch.float)
# build graph object
data = Data(x=x, x_additional=x_additional, edge_index=data_orig.edge_index)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
data

Data(x=[500, 1536], edge_index=[2, 2347], x_additional=[500, 36])

In [6]:
train_val_sets_kfold = generate_train_val_kfold(data,10)

In [7]:
train_val_sets_kfold

[(Data(x=[500, 1536], edge_label_index=[2, 235], edge_label=[235], x_additional=[500, 36]),
  Data(x=[500, 1536], edge_label_index=[2, 4224], edge_label=[4224], x_additional=[500, 36])),
 (Data(x=[500, 1536], edge_label_index=[2, 235], edge_label=[235], x_additional=[500, 36]),
  Data(x=[500, 1536], edge_label_index=[2, 4224], edge_label=[4224], x_additional=[500, 36])),
 (Data(x=[500, 1536], edge_label_index=[2, 235], edge_label=[235], x_additional=[500, 36]),
  Data(x=[500, 1536], edge_label_index=[2, 4224], edge_label=[4224], x_additional=[500, 36])),
 (Data(x=[500, 1536], edge_label_index=[2, 235], edge_label=[235], x_additional=[500, 36]),
  Data(x=[500, 1536], edge_label_index=[2, 4224], edge_label=[4224], x_additional=[500, 36])),
 (Data(x=[500, 1536], edge_label_index=[2, 235], edge_label=[235], x_additional=[500, 36]),
  Data(x=[500, 1536], edge_label_index=[2, 4224], edge_label=[4224], x_additional=[500, 36])),
 (Data(x=[500, 1536], edge_label_index=[2, 235], edge_label=[235]

In [8]:


train_val_sets = train_val_sets_kfold

results = []

for fold, (train_data, val_data) in enumerate(train_val_sets):
    print(f'Fold {fold + 1}:')
    
    # training
    model = train_model(
        train_data, 
        
        hidden_channels=64, 
        num_heads=16, 
        dropout=0.5, 
        lr=0.000005, 
        weight_decay=1e-3, 
        num_epochs=200, 
        print_interval=10
    )
    
    results.append({
        'fold': fold + 1,
       
        'model': model
    })
    
    print('--------------')



model_list = [r['model'] for r in results]
test_auc = [test_model(i, val_data) for i in model_list]
test_auc

Fold 1:
--------------
Fold 2:
--------------
Fold 3:
--------------
Fold 4:
--------------
Fold 5:
--------------
Fold 6:
--------------
Fold 7:
--------------
Fold 8:
--------------
Fold 9:
--------------
Fold 10:
--------------


[(np.float64(0.9422497781695589), np.float64(0.9380913099871461)),
 (np.float64(0.9322687555139428), np.float64(0.9415692054840911)),
 (np.float64(0.9271067770398809), np.float64(0.9329777195639909)),
 (np.float64(0.9399754745164731), np.float64(0.9493861832493462)),
 (np.float64(0.9253774891940774), np.float64(0.9255588587001925)),
 (np.float64(0.9481379066076442), np.float64(0.9619429359055307)),
 (np.float64(0.9363832455797686), np.float64(0.942159381593986)),
 (np.float64(0.9033548486118099), np.float64(0.9110622520448427)),
 (np.float64(0.9277950032414456), np.float64(0.9441822186097422)),
 (np.float64(0.9277621255293877), np.float64(0.9294655557767938))]